In [1]:
import helpers as hp
import nltk
import csv
from nltk.corpus import wordnet as wn
import preprocess 
from preprocess import pos_tag_db, lemmatize_db
import csv
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from re import search
# Get a list of available databases
dbs = hp.getDatabases()

0	EACH PORTION: 1 Apple

1	TEMPERATURE: 300° F. Oven

2	Pierce top of apple holding paring knife at a 45 degree angle to stem. Rotate knife around top to produce small cone-shaped top. Set tops aside. Using melon baller or teaspoon, remove core from apples, being careful not to pierce the bottom of apples. 

3	In small bowl, stir together tahini, raisins, lemon juice, grated lemon peel, maple syrup, cinnamon and a pinch of ground cardamom (optional). Fill apple cavitites almost to top with raisin mixture. Replace apple tops. Set apples in baking tray, pour orange juice over apples and bake for 15 minutes or until apples are soft when pierced with toothpick.

4	Saute mushrooms in 2 tablespoons of the stock, adding more stock if necessary, or in 1 tablespoon oil in large pot over medium heat for 5 minutes or until they start to brown.

5	Add remaining stock, green onions, marjoram, thyme, 1/4 teaspoon salt if desired, and pepper; simmer fro 10 minutes. Adjust seasoning. Divide broth amon

In [2]:
#creating a list of ingredients
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import sys
url = "https://world.openfoodfacts.org/ingredients.json"
page = urlopen(url)
html = page.read()
soup = BeautifulSoup(html) 
u = soup.decode('utf-8')
u = u.split('"name":')
#creating a csv file with ingredients
localFile = open('ingredients.csv', 'w')
#creating a list of ingredients
ingredients = []
for line in u:
    ingredients.append(line.split(',')[0])
del ingredients[0]
for ingredient in ingredients:
    localFile.write(ingredient)
localFile.close()


In [3]:
#we have 10000 ingredients in dataset
len(ingredients)

10000

## First database : 'VegetarianRecipes.exl'

In [4]:
#data structure of clean_db is a list of strings
clean_db = hp.cleanFile(dbs[0])
  
#merging the cleaned sentenes 
vegetarian_recipes = []
for sentence in clean_db:
    vegetarian_recipes.append(sentence)
#split into separate recipes
split_vegetarian_recipes = str(vegetarian_recipes).split('EACH')
del split_vegetarian_recipes[0]

In [13]:
#writing the recipes to a csv file
with open('recipes.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["EACH PORTION", 'TEMPERATURE', "METHOD", "NOTE", "INGREDIENTS"])
    #loop through each recipes 
    allmethod = []
    for recipe in split_vegetarian_recipes:
        #for each recipe create a list
        portion = []
        method = []
        temperature = []
        pan_size = []
        note = []
        contains = []
        for sent in re.split(',|\n', recipe):
            if search('PORTION:', sent):
                sent.replace('PORTION:','')
                sent.replace('PAN','')
                portion.append(sent.replace('PORTION:',''))
                    
            elif search('TEMPERATURE:', sent):
                temperature.append(sent.replace('TEMPERATURE:',''))
                
            elif search('PAN SIZE:', sent):
                pan_size.append(sent.replace('PAN SIZE:',''))
                    
            elif search('NOTE:', sent):
                note.append(sent.replace('NOTE:',''))
            else:
                method.append(sent)
        #merge method (as a list) into one single string
        method_str = ""
        for text in method:
            method_str += str(text) + " "
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        method_str = tokenizer.tokenize(method_str)
        #lemmatize and POS tag the method string
        #PoS-Tag, lemmatize and format
        postagged = pos_tag_db(method_str)
        lemmatized = lemmatize_db(postagged, [".", "X"])
        formatted = hp.formatFile(lemmatized)
        method_tagged = formatted
        onelist = []
        for pair in method_tagged:
            onelist.extend(pair)
        for (word, tag) in onelist:
            if (tag == 'NOUN'):
                contains.append(word)
        #from contains we can filter for features like ingredients and tools     
               
        #write row for each recipe
        writer.writerow([portion, temperature, onelist, note, contains])   

## Second database: 'data/EthnicRecipes.exl'

In [482]:
clean_db = hp.cleanFile(dbs[1])
#merging the cleaned sentenes 
ethnic_recipes = []
for sentence in clean_db:
    ethnic_recipes.append(sentence)
#split the recipes into lists
split_ethnic_recipes = str(ethnic_recipes).split('COOK')

In [483]:
#where is the method?
clean_db

['COOK TIME: 1 Hr 30 Min\n',
 'YIELDS: 6 Servings\n',
 'Serve these flavorful beans over rice, as a sauce for pasta or with a good loaf of bread.\n',
 'Copyright @ 2005\n',
 'www.allrecipes.com\n',
 'COOK TIME: 1 Hr 30 Min\n',
 'YIELDS: 6 Servings\n',
 'NOTE: Serve over rice.\n',
 'Copyright @ 2005\n',
 'www.allrecipes.com\n',
 'COOK TIME: 30 Min\n',
 'YIELDS: 4 Servings\n',
 'NOTE: You may substitute cashews for peanuts, or bamboo shoots for the water chestnuts.\n',
 'NOTE: The nutrition data for this recipe includes information for the full amount of the marinade ingredients. Depending on marinating time, ingredients, cooking method, etc., the actual amount of the marinade consumed will vary.\n',
 'Copyright @ 2005\n',
 'www.allrecipes.com\n',
 'COOK TIME: 2 Hr\n',
 'YIELDS: 6 Servings\n',
 'NOTE: Serve with plain yogurt and a salad to mix with the rice dish.\n',
 'NOTE: You may substitute 1 large eggplant for cauliflower or use both. Slice eggplant in rounds, cut larger pieces in ha

## Third database: data/ArmedForcesRecipes.exl'

In [493]:
clean_db = hp.cleanFile(dbs[2])
armedforces_recipes = []
for sentence in clean_db:
    armedforces_recipes.append(sentence)
#split the recipes into lists
split_armedforces_recipes = str(armedforces_recipes).split('EACH')

In [494]:
#writing the recipes to a csv file
with open('recipes.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    #writer.writerow(["EACH PORTION", 'TEMPERATURE', "METHOD", "NOTE"])
    #loop through each recipes  
    for i in range(1, len(split_armedforces_recipes)): 
        #for each recipe create a list
        portion = []
        method = []
        temperature = []
        pan_size = []
        note = []
        for sent in re.split(',|\n', split_armedforces_recipes[i]):
            if search('PORTION:', sent):
                sent.replace('PORTION:','')
                sent.replace('PAN','')
                portion.append(sent.replace('PORTION:',''))
                    
            elif search('TEMPERATURE:', sent):
                temperature.append(sent.replace('TEMPERATURE:',''))
                
            elif search('PAN SIZE:', sent):
                pan_size.append(sent.replace('PAN SIZE:',''))
                    
            elif search('NOTE:', sent):
                note.append(sent.replace('NOTE:',''))
            else:
                method.append(sent)
    
    writer.writerow([portion, temperature, method, note])   

## Fourth database: 'data/CommonRecipes.exl'

In [495]:
clean_db = hp.cleanFile(dbs[3])
common_recipes = []
for sentence in clean_db:
    common_recipes.append(sentence)
#split the recipes into lists
split_common_recipes = str(common_recipes).split('EACH')

In [505]:
#writing the recipes to a csv file
with open('recipes.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    #writer.writerow(["EACH PORTION", 'TEMPERATURE', "METHOD", "NOTE"])
    #loop through each recipes  
    for i in range(1, len(split_armedforces_recipes)): 
        #for each recipe create a list
        portion = []
        method = []
        temperature = []
        pan_size = []
        note = []
        yild = []
        for sent in re.split(',|\n', split_armedforces_recipes[i]):
            if search('PORTION:', sent):
                sent.replace('PORTION:','')
                sent.replace('PAN','')
                portion.append(sent.replace('PORTION:',''))
                    
            elif search('TEMPERATURE:', sent):
                temperature.append(sent.replace('TEMPERATURE:',''))
                
            elif search('PAN SIZE:', sent):
                pan_size.append(sent.replace('PAN SIZE:',''))
               
            elif search('YIELD:', sent):
                yild.append(sent.replace('YIELD:',''))
                    
            elif search('NOTE:', sent):
                note.append(sent.replace('NOTE:',''))
            else:
                method.append(sent)
    
    writer.writerow([portion, temperature, method, note])   